In [1]:
import json
import os
import glob
import pandas as pd
import numpy as np
from collections import Counter, defaultdict
import random

import matplotlib.pyplot as plt
import seaborn as sns
sns.set(style="whitegrid", palette="pastel")

In [3]:
DISK_IO_DIR = "/jagupard19/scr1/yiweil"
if not os.path.exists(DISK_IO_DIR):
    os.mkdir(DISK_IO_DIR)

# Is contentiousness correlated w/ virality for Reddit posts?

Ways of measuring contentiousness:
- percentage of disagreeing comments (according to Zhang et al.-trained BERT)
- percentage of 

Ways of measuring virality:
- num likes
- upvotes


## r/changemyview

In [5]:
# For each comment, label as agreeing or disagreeing
comment_labels_df = pd.read_csv(os.path.join(DISK_IO_DIR,'cmv_comments_with_predictions.csv'),sep=',',header=0)
# comments_df = pd.read_csv('/u/scr/yiweil/Green-American-Lexicon/1_data_collection/reddit/\
# comments_output/changemyview/from_posts_1-1-2010_to_9-22-2021.csv',sep=',')
# comment_id2parent_id = dict(zip(comments_df['id'],comments_df['parent_id']))

# Iterate over comments w/ labels
# Increment counter of parent post's num agreeing and disagreeing comments
post_id2agreement = defaultdict(lambda: defaultdict(int))
for ix,row in comment_labels_df.iterrows():
    c_id = row['id']
    comment_label = row['predicted']
    parent_post_id = comment_id2parent_id[c_id]
    post_id2agreement[parent_post_id][comment_label] += 1

# posts_df = pd.read_csv('/u/scr/yiweil/Green-American-Lexicon/1_data_collection/reddit/\
# pmaw_output/submissions/changemyview/1-1-2010_to_09-22-2021.csv',index_col=0)
# Then get percentages for each post
posts_df['n_agree_comments'] = posts_df['id'].apply(lambda x: post_id2agreement[x]['agreement'])
posts_df['n_disagree_comments'] = posts_df['id'].apply(lambda x: post_id2agreement[x]['disagreement'])
posts_df['pct_agree_comments'] = posts_df.apply(lambda x: x['n_agree_comments']/x['num_comments'],axis=1)
posts_df['pct_disagree_comments'] = posts_df.apply(lambda x: x['n_disagree_comments']/x['num_comments'],axis=1)

# Plotting/correlation testing

In [6]:
sns.scatterplot(data=posts_df,x='n_agree_comments',y='upvote_ratio')

defaultdict(<function __main__.<lambda>()>,
            {'dsfjsdf': defaultdict(int, {'agreement': 1})})

In [ ]:
sns.scatterplot(data=posts_df,x='pct_agree_comments',y='upvote_ratio')

In [7]:
sns.scatterplot(data=posts_df,x='n_disagree_comments',y='upvote_ratio')

defaultdict(int, {'agreement': 1})

In [ ]:
sns.scatterplot(data=posts_df,x='pct_disagree_comments',y='upvote_ratio')

In [8]:
sns.scatterplot(data=posts_df,x='n_agree_comments',y='n_disagree_comments')

array(['1vxyf3', '1vps5i', '1vgrxa'], dtype=object)

In [10]:
posts_df.columns

Index(['author', 'author_created_utc', 'author_flair_css_class',
       'author_flair_text', 'author_fullname', 'created_utc', 'domain',
       'edited', 'full_link', 'gilded', 'id', 'is_self', 'media_embed',
       'num_comments', 'over_18', 'permalink', 'retrieved_on', 'score',
       'secure_media_embed', 'selftext', 'stickied', 'subreddit',
       'subreddit_id', 'thumbnail', 'title', 'url', 'link_flair_css_class',
       'link_flair_text', 'post_hint', 'preview', 'locked', 'banned_by',
       'keyword', 'contest_mode', 'spoiler', 'brand_safe', 'suggested_sort',
       'approved_at_utc', 'banned_at_utc', 'can_mod_post', 'is_video',
       'is_crosspostable', 'is_reddit_media_domain', 'num_crossposts',
       'parent_whitelist_status', 'pinned', 'whitelist_status',
       'thumbnail_height', 'thumbnail_width', 'view_count', 'subreddit_type',
       'author_flair_richtext', 'author_flair_type', 'link_flair_richtext',
       'link_flair_text_color', 'link_flair_type', 'rte_mode',
    